In [13]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import zipfile
from os import path, getcwd, chdir

In [2]:
!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

--2020-04-06 09:58:38--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.26.16, 2404:6800:4004:813::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.26.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  6.89MB/s    in 0.4s    

2020-04-06 09:58:39 (6.89 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [16]:
zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/happy-or-sad")
zip_ref.close()

In [33]:
def train_happy_sad_model():
    DESIRED_ACCURACY = 0.999

    class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if( logs.get('acc') >= DESIRED_ACCURACY):
                self.model.stop_training = True

    callbacks = myCallback()
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150,150,3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.Dense(1, activation='softmax'),
    ])
    
    model.compile(loss='binary_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['acc'])
    
    train_datagen = ImageDataGenerator(rescale=1/255)
    train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s',
        target_size=(150,150),
        batch_size=15,
        class_mode='binary')
    
    history = model.fit_generator(
      train_generator,
      steps_per_epoch=5,  
      epochs=20,
      verbose=1,callbacks=[callbacks])
    
    return history.history['acc'][-1]

In [34]:
train_happy_sad_model()

Found 80 images belonging to 2 classes.
  ...
    to  
  ['...']
Train for 5 steps
Epoch 1/20
5/5 [==============================] - 1s 112ms/step - loss: 7.5230 - acc: 0.5067
Epoch 2/20
5/5 [==============================] - 0s 23ms/step - loss: 7.8983 - acc: 0.4923
Epoch 3/20
5/5 [==============================] - 0s 22ms/step - loss: 7.6898 - acc: 0.5077
Epoch 4/20
5/5 [==============================] - 0s 24ms/step - loss: 7.5230 - acc: 0.5067
Epoch 5/20
5/5 [==============================] - 0s 25ms/step - loss: 8.1069 - acc: 0.4769
Epoch 6/20
5/5 [==============================] - 0s 23ms/step - loss: 8.1069 - acc: 0.4769
Epoch 7/20
5/5 [==============================] - 0s 21ms/step - loss: 7.8983 - acc: 0.4923
Epoch 8/20
5/5 [==============================] - 0s 24ms/step - loss: 7.6898 - acc: 0.5077
Epoch 9/20
5/5 [==============================] - 0s 23ms/step - loss: 7.6898 - acc: 0.5077
Epoch 10/20
5/5 [==============================] - 0s 26ms/step - loss: 7.6898 - acc: 0.

0.50769234